In [ ]:
import pathlib
import pandas as pd

datadir = pathlib.Path.cwd().parent / "data"

pd_hotel       = pd.read_parquet(datadir / "hotel.parquet")
pd_customer    = pd.read_parquet(datadir / "customer.parquet")
pd_reservation = pd.read_parquet(datadir / "reservation.parquet")
pd_campaign    = pd.read_parquet(datadir / "campaign.parquet")

# 15章 演習問題
## 15-1 多次元分析の前処理
### Q: 予約履歴の多次元分析のためのキューブ作成


In [ ]:
import numpy as np

(
    # （1） reservationから必要なデータを抽出
    pd_reservation
    # （1）-1 キャンセルを除外
    .query("status != 'canceled'")
    # （1）-2 2019年のデータを抽出
    .query("checkout_date.dt.year == 2019")
    .assign(
        # （1）-3 checkout_dateから年と月を抽出
        checkout_year=lambda df: df.checkout_date.dt.year,
        checkout_month=lambda df: df.checkout_date.dt.month,
        # （1）-4 campaignと結合するために予約日時を丸めて予約日の列を作成
        reserve_date=lambda df: df.reserved_at.dt.normalize()
    )
    # （2） customerと結合
    .merge(
        pd_customer
        # （2）-1 年齢を10刻みでカテゴリ化、60以上は一つのカテゴリにまとめる
        .assign(age_cat=lambda df:
            (df.age / 10)
            .astype(int).astype("category").cat.as_ordered()
            .mask(lambda s: s >= 6, 6))
        [["customer_id", "sex", "age_cat"]],
        on="customer_id", how="left"
    )
    # （3） campaignと結合
    .merge(
        pd_campaign
        # （3）-1 キャンペーン期間内の日をすべて列挙
        .assign(reserve_date=lambda df:
            df.apply(lambda r: pd.date_range(r.starts_at, r.ends_at), axis=1))
        [["campaign_name", "reserve_date"]]
        .explode("reserve_date"),
        on="reserve_date", how="left"
    )
    # （3）-2 結合用のカラムを削除
    .drop(columns="reserve_date")
    # （4） ディメンションをキーとしてreservationを集約
    .groupby(["checkout_year", "checkout_month", "hotel_id", "sex", "age_cat",
        "campaign_name"], observed=True, dropna=False)
    .agg(
        sales=("total_price", "sum"),
        reservation_cnt=("reservation_id", "size"),
        length_of_stay_avg=("length_of_stay", "mean"),
        people_num_avg=("people_num", "mean"),
    )
    .reset_index()
    # （5） 集約データにhotelを結合
    .merge(
        pd_hotel
        # （5）-1 unit_priceをカテゴリ化
        .assign(unit_price_range=lambda df: np.where(df.unit_price < 5000, 0,
                                            np.where(df.unit_price < 10000, 5000,
                                            np.where(df.unit_price < 20000, 10000,
                                            np.where(df.unit_price < 30000, 20000,
                                                    30000)))))
        [["hotel_id", "hotel_name", "hotel_type", "address_prefecture",
          "address_town", "unit_price_range"]],
        on="hotel_id", how="left"
    )
)

## 15-2 レコメンデーションの前処理
### Q: ホテルのレコメンデーション用のレーティング行列の作成


In [ ]:
from scipy.sparse import csr_matrix

# （1） 顧客・ホテルごとの予約数を集計した縦持ちデータを作成
long_df = (
    pd_reservation
    .groupby(["customer_id", "hotel_id"]).size().rename("cnt")
    .reset_index()
)

# （2） スパースマトリックスの行／列にする列をカテゴリ型に変換
customer_id = pd.Categorical(long_df["customer_id"])
hotel_id = pd.Categorical(long_df["hotel_id"])

# （3） 行と列のラベルを作成
row_label = customer_id.categories.to_numpy()
col_label = hotel_id.categories.to_numpy()

# （4） スパースマトリックス変換
csr_matrix((long_df["cnt"], (customer_id.codes, hotel_id.codes)))

## 15-3 予測モデルの前処理
### Q: 予測モデル作成のためのデータの作成


In [ ]:
#（1）BigQueryの特徴量テーブルから、顧客ごとに過去1年以内のある1ヶ月の特徴量をランダムに
#    サンプリング（学習データとする月の選定）して取得

# 顧客ごとに過去1年以内のある1ヶ月の特徴量をランダムにサンプリングするクエリ
sql = """
select
    * except (rn)
from (
    select
        *,
        row_number() over (partition by customer_id order by rand()) as rn
    from example.monthly_customer_feature
    where left(rsv_month, 4) = "2019"
)
where rn = 1
"""
# クエリを用いてBigQueryからデータを取得
# 事前にGCPの認証が必要で、認証方法は環境によって異なる
# colabの場合:
#   セルで下記を実行
#     from google.colab import auth
#     auth.authenticate_user()
# PC等のローカル環境の場合:
#   https://cloud.google.com/sdk/docs/install-sdk からgcloud CLIをインストールし、下記を実行
#     gcloud auth application-default login
from google.cloud import bigquery
project_id = "GCPのプロジェクトID"
client = bigquery.Client(project=project_id)
raw_feature = client.query(sql).to_dataframe()

#（2）カテゴリ化やエンコーディング、数値変換などのモデリングのための特徴量加工
import numpy as np
from category_encoders.one_hot import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

# エンコーディングや数値変換を行うオブジェクトを生成
onehot_encoder = OneHotEncoder(cols="sex", use_cat_names=True)
log_scaler = FunctionTransformer(func=np.log1p)

# 特徴量加工
feature = (
    raw_feature
    .assign(
        # 月を数値化
        month=lambda df: df.rsv_month.str[-2:].astype(int),
        # 年齢を10区切りでカテゴリ化、60以上は1カテゴリにまとめる
        age=lambda df:
            (df.age / 10)
            .astype(int).astype("category").cat.as_ordered()
            .mask(lambda s: s >= 6, 6),
        # 過去1年間の合計金額を対数化
        log_total_price_last_year=lambda df:
            log_scaler.transform(df["total_price_last_year"])
    )
    # sexをone-hot encoding
    .pipe(onehot_encoder.fit_transform)
    # 不要な列を削除
    .drop(columns=["rsv_month", "customer_id", "total_price_last_year"])
)

#（3）ホールドアウト検証用のテストデータを取り出し、残ったデータを交差検定用に分割
from sklearn.model_selection import train_test_split, KFold

# ホールドアウト検証用のデータ分割
train_data, test_data = train_test_split(
    feature, test_size=0.2, shuffle=True, random_state=71)

# 交差検証用のデータ分割方法を定義
fold = KFold(n_splits=4, shuffle=True, random_state=71)

# 分割数分の繰り返し処理（並列処理も可能）
for train_cv_no, test_cv_no in fold.split(train_data):

    # 交差検証における学習データ、テストデータを抽出
    train_cv = train_data.iloc[train_cv_no]
    test_cv = train_data.iloc[test_cv_no]

    # ここにtrain_cvを学習データ、test_cvをテストデータとした機械学習モデルの学習、検証処理を書く

# ここに交差検証の結果をまとめる処理を書く